In [1]:
import numpy as np
import pandas as pd
from os.path import join

# below code removes extra commas in column K related to profession:
the output file will have the same name as the input, but including '_' at the beginning

In [2]:
# # for small files
# input_file_header = './../header.txt'
# input_folder = './../Data/Dataton_train/Splitted/'
# input_file_data_lista = [
#     'Dataton_train_1.csv', 'Dataton_train_2.csv', 'Dataton_train_3.csv',
#     'Dataton_train_4.csv', 'Dataton_train_5.csv', 'Dataton_train_6.csv', 'Dataton_train_7.csv',
#     'Dataton_train_8.csv', 'Dataton_train_9.csv', 'Dataton_train_10.csv', 'Dataton_train_11.csv'
#     ]
# output_folder = './../Data/Dataton_train/Splitted/'

# whole dataset
input_file_header = './header.txt'
input_folder = './../Data/Dataton_train/'
input_file_data_lista = ['Dataton_train.csv']
output_folder = './../Data/Dataton_train/'

for ix,input_file_data in enumerate(input_file_data_lista):
    output_file = "_"+input_file_data
        
    # create file
    text_file = open(output_folder + output_file, "w")
    n = text_file.write("")
    text_file.close()
    
    # open file to append string
    text_file = open(output_folder + output_file, "wt")
    
    outputString = ""
    with open(input_folder+input_file_data) as fp:
        line = fp.readline()
        cnt = 0
        while line:
            # correcting length of lines by extra commas
            stringLength = line.count(',')
            if stringLength > 64:
                nDiff = stringLength - 64
                lineL = list(line)
                count = 0
                for ix, val in enumerate(lineL):
                    if val == ',':
                        if count > 6 and count <= 6 + nDiff:
                            lineL[ix] = "-"
                        count += 1
                line = "".join(lineL)
            # append to output
            outputString = outputString + line
            # read next line
            line = fp.readline()
            cnt += 1
            # write into file every 10.000 lines
            if (cnt > 10000):
                n = text_file.write(outputString)
                outputString = ""

    # close file at the end
    text_file.close()
    print('ready: {}'.format(output_file))

ready: _Dataton_train.csv


# I will read every dataset file, process it and save it using the expected format variables. 

# Reformating data in one step (to understand step by step go to next section)
### output files will have the same names. Thus, if output folder is the same, file will be rewritten

In [1]:
import numpy as np
import pandas as pd

# # for small files
# input_file_header = './../header.txt'
# input_folder = './../Data/Dataton_train/Splitted/'
# input_file_data_lista = [
#     '_Dataton_train_1.csv', '_Dataton_train_2.csv', '_Dataton_train_3.csv',
#     '_Dataton_train_4.csv', '_Dataton_train_5.csv', '_Dataton_train_6.csv', '_Dataton_train_7.csv',
#     '_Dataton_train_8.csv', '_Dataton_train_9.csv', '_Dataton_train_10.csv', '_Dataton_train_11.csv'
#     ]
# output_folder = input_folder

# whole dataset
input_file_header = './header.txt'
input_folder = './../Data/Dataton_train/'
input_file_data_lista = ['_Dataton_train.csv']
output_folder = input_folder

# variables to check
cols_to_check = [
    'edad',
     'categoria',
     'mora_max',
     'cant_moras_30_ult_12_meses',
     'cant_moras_60_ult_12_meses',
     'cant_moras_90_ult_12_meses',
     'nro_tot_cuentas',
     'ctas_activas',
     'ctas_embargadas',
     'cuota_cred_hipot',
     'ingreso_final',
     'cant_mora_30_tdc_ult_3m_sf',
     'cant_mora_30_consum_ult_3m_sf',
     'cant_oblig_tot_sf',
     'cant_cast_ult_12m_sr',
     'ind',
     'pol_centr_ext',
     'ingreso_nomina',
     'ingreso_segurida_social'
]

listVarIntegers = [
     'categoria',
     'mora_max',
     'cant_moras_30_ult_12_meses',
     'cant_moras_60_ult_12_meses',
     'cant_moras_90_ult_12_meses',
     'nro_tot_cuentas',
     'ctas_activas',
     'ctas_embargadas',
     'ingreso_segurida_social',

]

for ix,input_file_data in enumerate(input_file_data_lista):
    
    output_file = "_"+input_file_data

    # reading headers
    header_cols = np.array( pd.read_csv(input_folder + input_file_header, header = None) )[0]
    
    # reading whole data file 
    # df = pd.read_csv( input_folder + input_file_data, header = None, sep =",")
    
    # reading data
    firstTime = True
    for df in pd.read_csv(input_folder + input_file_data, iterator=True, chunksize=1000000):
            
        # adding header names to dataframe
        df.columns = ( header_cols )

        # try to convert data to numeric,
        # if it fails, then look for '\\N' to transform it into np.nan.
        # finally try again to convert data to numeric format.
        # it prints each category with the number of erroneous data
        for col in cols_to_check:
            try:
                if col in listVarIntegers:
                    # as integers
                    df[col] = pd.to_numeric( df[col] , downcast = 'integer' )
                else:
                    # as float
                    df[col] = pd.to_numeric( df[col] , downcast = 'float' )
            except:
                var = df[col].to_numpy() # var is pointing to dataframe df
                count = 0
                for ix, item in enumerate(var):
                    if not (isinstance(item, float)):
                        if (item == '\\N'):
                            var[ix] = np.nan
                            count += 1                      
                #print('{} : {}'.format( col, count) )
            if col in listVarIntegers:
                # as integers
                df[col] = pd.to_numeric( df[col] , downcast = 'integer' )
            else:
                # as float
                df[col] = pd.to_numeric( df[col] , downcast = 'float' )

        # formating float64 to 32 bits
        for k in range( len(df.columns) ):
            if str(df.iloc[:,k].dtypes) == 'float64':
                df.iloc[:,k] = pd.to_numeric( df.iloc[:,k] , downcast = 'float' )

        if (firstTime):
            # saving results
            df.to_csv(output_folder + output_file, sep = ',', index = False, header = False)
            firstTime = False
        else:
            df.to_csv(output_folder + output_file, sep = ',', index = False, header = False, mode='a')
            
print('ready: {}'.format(output_file) )

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (3,11,29,30,32,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (3,29,30,32,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ready: __Dataton_train.csv


# This is step by step. To run all in one cell for various files go to next section

In [48]:
input_file_data = '_Dataton_train_1.csv'
input_file_header = './../header.txt'
input_folder = './../Data/Dataton_train/Splitted/'

output_file = input_file_data
output_folder = input_folder

In [3]:
# reading data
df = pd.read_csv( input_folder + input_file_data, header = None, sep =",")
# reading headers
header_cols = np.array( pd.read_csv(input_folder + input_file_header, header = None) )[0]
# adding header names to dataframe
df.columns = ( header_cols )

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (3,29,30,32,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


about above result: The reason you get this low_memory warning is because guessing dtypes for each column is very memory demanding. Pandas tries to determine what dtype to set by analyzing the data in each column. (https://stackoverflow.com/questions/24251219/pandas-read-csv-low-memory-and-dtype-options)

In [4]:
df

,periodo,id_cli,fecha_nacimiento,edad,genero,estado_civil,nivel_academico,profesion,ocupacion,tipo_vivienda,...,cuota_libranza_sf,cant_oblig_tot_sf,cant_cast_ult_12m_sr,ind,rep_calif_cred,pol_centr_ext,convenio_lib,ingreso_nomina,ingreso_segurida_social,gasto_familiar
0,201908,2089776,19840630,35.0582,M,DIVORCIADO,TECNOLOGO,TECNOLOGIA SISTEMAS,Empleado,ALQUILADA,...,0.0,1,0,311715.5,C,0,\N,1255032,\N,232526.0
1,201909,2089776,19840630,35.1431,M,DIVORCIADO,TECNOLOGO,TECNOLOGIA SISTEMAS,Empleado,ALQUILADA,...,0.0,1,0,311715.5,C,0,\N,1255032,\N,265900.0
2,201907,2089776,19840630,34.9624,M,DIVORCIADO,TECNOLOGO,TECNOLOGIA SISTEMAS,Empleado,ALQUILADA,...,0.0,1,0,311306,C,0,\N,1255032,\N,243052.0
3,201903,2089776,19840630,34.6393,M,DIVORCIADO,TECNOLOGO,TECNOLOGIA SISTEMAS,Empleado,ALQUILADA,...,0.0,1,0,311306,C,0,\N,1172612,\N,276014.0
4,201911,2089776,19840630,35.3073,M,DIVORCIADO,TECNOLOGO,TECNOLOGIA SISTEMAS,Empleado,ALQUILADA,...,0.0,0,0,677516,C,0,\N,1255032,\N,418866.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2089651,202008,1823154,19830618,37.0951,M,SOLTERO,NO INFORMA,\N,Empleado,FAMILIAR,...,0.0,2,0,848790,G,4,\N,\N,\N,335692.0
2089652,202001,87167,19821202,37.0376,M,NO INFORMA,NO INFORMA,\N,Independiente,ALQUILADA,...,0.0,\N,\N,4.36016e+06,SIN INFO,\N,\N,\N,\N,112700.0
2089653,202008,87167,19821202,37.6372,M,NO INFORMA,NO INFORMA,\N,Independiente,ALQUILADA,...,0.0,\N,\N,6.30046e+06,SIN INFO,\N,\N,\N,\N,510292.0
2089654,201911,87167,19821202,36.8843,M,NO INFORMA,NO INFORMA,\N,Independiente,ALQUILADA,...,0.0,\N,\N,5.00514e+06,SIN INFO,\N,\N,\N,\N,61900.0


In [5]:
for k in range( len(df.columns) ):
    print( '{} \t:\t {}'.format( df.iloc[:,k].dtype , header_cols[k] ))

int64 	:	 periodo
int64 	:	 id_cli
int64 	:	 fecha_nacimiento
object 	:	 edad
object 	:	 genero
object 	:	 estado_civil
object 	:	 nivel_academico
object 	:	 profesion
object 	:	 ocupacion
object 	:	 tipo_vivienda
int64 	:	 ult_actual
object 	:	 categoria
int64 	:	 codigo_ciiu
object 	:	 ind_mora_vigente
object 	:	 cartera_castigada
object 	:	 ciudad_residencia
object 	:	 departamento_residencia
object 	:	 ciudad_laboral
object 	:	 departamento_laboral
object 	:	 rechazo_credito
object 	:	 mora_max
object 	:	 cant_moras_30_ult_12_meses
object 	:	 cant_moras_60_ult_12_meses
object 	:	 cant_moras_90_ult_12_meses
float64 	:	 cupo_total_tc
object 	:	 tenencia_tc
float64 	:	 cuota_tc_bancolombia
object 	:	 tiene_consumo
object 	:	 tiene_crediagil
object 	:	 nro_tot_cuentas
object 	:	 ctas_activas
object 	:	 tiene_ctas_activas
object 	:	 ctas_embargadas
object 	:	 tiene_ctas_embargadas
object 	:	 pension_fopep
object 	:	 cuota_cred_hipot
object 	:	 tiene_cred_hipo_1
object 	:	 tiene_cred_hip

In [6]:
cols_to_check = [
    'edad',
     'categoria',
     'mora_max',
     'cant_moras_30_ult_12_meses',
     'cant_moras_60_ult_12_meses',
     'cant_moras_90_ult_12_meses',
     'nro_tot_cuentas',
     'ctas_activas',
     'ctas_embargadas',
     'cuota_cred_hipot',
     'ingreso_final',
     'cant_mora_30_tdc_ult_3m_sf',
     'cant_mora_30_consum_ult_3m_sf',
     'cant_oblig_tot_sf',
     'cant_cast_ult_12m_sr',
     'ind',
     'pol_centr_ext',
     'ingreso_nomina',
     'ingreso_segurida_social'
]

listVarIntegers = [
     'categoria',
     'mora_max',
     'cant_moras_30_ult_12_meses',
     'cant_moras_60_ult_12_meses',
     'cant_moras_90_ult_12_meses',
     'nro_tot_cuentas',
     'ctas_activas',
     'ctas_embargadas',
     'ingreso_segurida_social',

]

* Changing \\n for nan values:

In [7]:
# try to convert data to numeric,
# if it fails, then look for '\\N' to transform it into np.nan.
# finally try again to convert data to numeric format.
# it prints each category with the number of erroneous data
for col in cols_to_check:
    try:
        if col in listVarIntegers:
            # as integers
            df[col] = pd.to_numeric( df[col] , downcast = 'integer' )
        else:
            # as float
            df[col] = pd.to_numeric( df[col] , downcast = 'float' )
    except:
        var = df[col].to_numpy() # var is pointing to dataframe df
        count = 0
        for ix, item in enumerate(var):
            if not (isinstance(item, float)):
                if (item == '\\N'):
                    var[ix] = np.nan
                    count += 1                      
        print('{} : {}'.format( col, count) )
    if col in listVarIntegers:
        # as integers
        df[col] = pd.to_numeric( df[col] , downcast = 'integer' )
    else:
        # as float
        df[col] = pd.to_numeric( df[col] , downcast = 'float' )

edad : 525
categoria : 22819
mora_max : 593247
cant_moras_30_ult_12_meses : 593247
cant_moras_60_ult_12_meses : 593247
cant_moras_90_ult_12_meses : 593247
nro_tot_cuentas : 10
ctas_activas : 10
ctas_embargadas : 10
cuota_cred_hipot : 1927888
ingreso_final : 15215
cant_mora_30_tdc_ult_3m_sf : 326473
cant_mora_30_consum_ult_3m_sf : 326473
cant_oblig_tot_sf : 338751
cant_cast_ult_12m_sr : 338751
ind : 696
pol_centr_ext : 306909
ingreso_nomina : 1540662
ingreso_segurida_social : 1508065


* print types again:

In [8]:
for k in range( len(df.columns) ):
    print( '{} \t:\t {}'.format( df.iloc[:,k].dtype , header_cols[k] ))

int64 	:	 periodo
int64 	:	 id_cli
int64 	:	 fecha_nacimiento
float32 	:	 edad
object 	:	 genero
object 	:	 estado_civil
object 	:	 nivel_academico
object 	:	 profesion
object 	:	 ocupacion
object 	:	 tipo_vivienda
int64 	:	 ult_actual
float64 	:	 categoria
int64 	:	 codigo_ciiu
object 	:	 ind_mora_vigente
object 	:	 cartera_castigada
object 	:	 ciudad_residencia
object 	:	 departamento_residencia
object 	:	 ciudad_laboral
object 	:	 departamento_laboral
object 	:	 rechazo_credito
float64 	:	 mora_max
float64 	:	 cant_moras_30_ult_12_meses
float64 	:	 cant_moras_60_ult_12_meses
float64 	:	 cant_moras_90_ult_12_meses
float64 	:	 cupo_total_tc
object 	:	 tenencia_tc
float64 	:	 cuota_tc_bancolombia
object 	:	 tiene_consumo
object 	:	 tiene_crediagil
float64 	:	 nro_tot_cuentas
float64 	:	 ctas_activas
object 	:	 tiene_ctas_activas
float64 	:	 ctas_embargadas
object 	:	 tiene_ctas_embargadas
object 	:	 pension_fopep
float32 	:	 cuota_cred_hipot
object 	:	 tiene_cred_hipo_1
object 	:	 tien

> integer variables were not returned as int because it contains nan values. 
Depending on the variable meaning, it could be possible to transform nan values into a numeric value 
by using :
<br>
_dataframe.fillna(newValue)_

* in order to make an efficient memory usage, every float64 data will be transformed into float32:

In [43]:
for k in range( len(df.columns) ):
    if str(df.iloc[:,k].dtypes) == 'float64':
        df.iloc[:,k] = pd.to_numeric( df.iloc[:,k] , downcast = 'float' )

* print types again:

In [45]:
for k in range( len(df.columns) ):
    print( '{} \t:\t {}'.format( df.iloc[:,k].dtype , header_cols[k] ))

int64 	:	 periodo
int64 	:	 id_cli
int64 	:	 fecha_nacimiento
float32 	:	 edad
object 	:	 genero
object 	:	 estado_civil
object 	:	 nivel_academico
object 	:	 profesion
object 	:	 ocupacion
object 	:	 tipo_vivienda
int64 	:	 ult_actual
float32 	:	 categoria
int64 	:	 codigo_ciiu
object 	:	 ind_mora_vigente
object 	:	 cartera_castigada
object 	:	 ciudad_residencia
object 	:	 departamento_residencia
object 	:	 ciudad_laboral
object 	:	 departamento_laboral
object 	:	 rechazo_credito
float32 	:	 mora_max
float32 	:	 cant_moras_30_ult_12_meses
float32 	:	 cant_moras_60_ult_12_meses
float32 	:	 cant_moras_90_ult_12_meses
float32 	:	 cupo_total_tc
object 	:	 tenencia_tc
float32 	:	 cuota_tc_bancolombia
object 	:	 tiene_consumo
object 	:	 tiene_crediagil
float32 	:	 nro_tot_cuentas
float32 	:	 ctas_activas
object 	:	 tiene_ctas_activas
float32 	:	 ctas_embargadas
object 	:	 tiene_ctas_embargadas
object 	:	 pension_fopep
float32 	:	 cuota_cred_hipot
object 	:	 tiene_cred_hipo_1
object 	:	 tien

* saving data into files: output files will be the same as the input files:

In [49]:
df.to_csv(output_folder + output_file, sep = ',', index = False, header = False)
print('ready: {}'.format(output_file) )

# reading final dataset

In [ ]:
import numpy as np
import pandas as pd

# # for small files
# input_file_header = './../header.txt'
# input_folder = './../Data/Dataton_train/Splitted/'
# input_file_data_lista = [
#     '_Dataton_train_1.csv', '_Dataton_train_2.csv', '_Dataton_train_3.csv',
#     '_Dataton_train_4.csv', '_Dataton_train_5.csv', '_Dataton_train_6.csv', '_Dataton_train_7.csv',
#     '_Dataton_train_8.csv', '_Dataton_train_9.csv', '_Dataton_train_10.csv', '_Dataton_train_11.csv'
#     ]

# whole dataset
input_file_header = './header.txt'
input_folder = './../Data/Dataton_train/'
input_file_data_lista = ['_Dataton_train.csv']

for ix,input_file_data in enumerate(input_file_data_lista):
    
    output_file = "_"+input_file_data

    # reading headers
    header_cols = np.array( pd.read_csv(input_folder + input_file_header, header = None) )[0]
    
    # reading whole data file 
    # df = pd.read_csv( input_folder + input_file_data, header = None, sep =",")
    
    # reading data
    firstTime = True
    for df in pd.read_csv(input_folder + input_file_data, iterator=True, chunksize=1000000):
